# Visualisierung mit `pyLDAvis`

Im letzten Teil hast du LDA als Topic Model ausprobiert. Die Ergebnisse waren nicht so gut interpretierbar, daher interessierst du dich für deren Struktur, um evtl. etwas verbessern zu können.

Dabei unterstützt dich `pyLDAvis` als Visualisierungstool.

### ACHTUNG Colab-User

Du musst nach der Installation von pyLDAvis die Runtime-Umgebung neu starten, sonst funktioniert das nicht!

In [ ]:
!pip install pyLDAvis

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

Anschließend führst du die Vektorisierung durch. Das kann auch wieder einen Augenblick dauern:

In [ ]:
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(stop_words=stop_words, min_df=5)
count_vectors = count_vectorizer.fit_transform(df["nav"])
count_vectors

Durch die Nutzung von `min_df=5` ist die Matrix einigermaßen übersichtlich geblieben!

## Topic Model mit LDA berechnen

Die Aufrufsyntax des Topic Models ist sehr ähnlich zu der des `TfidfVectorizers`, nur dass du hier keine Transformation durchführen musst, daher heißt die Methode nur `fit`. Der Aufruf kann ein paar Sekunden dauern:

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

num_topics = 10

lda = LatentDirichletAllocation(n_components = num_topics, random_state=42)
lda.fit(count_vectors)

Um die Topics übersichtlich darstellen zu können, kannst du eine Funktion nutzen, die einen separaten Dataframe dafür aufbaut. Dazu iterierst du über alle Topics (`n_components` im Topic Model) und ermittelst jeweils immer die wichtigsten Wörter. `argsort` sortiert aufsteigend, daher sind es die letzten Indizes im Array:

In [ ]:
def topics_table(model, feature_names, n_top_words = 20):
    word_dict = {}
    
    for i in range(model.n_components):
        # ermittle für jedes Topic die größten Werte
        words_ids = model.components_[i].argsort()[:-n_top_words-1:-1]
        words = [feature_names[key] for key in words_ids]
        # und füge die entsprechenden Worte im Klartext dem Dictionary hinzu
        word_dict['Topic #%02d' % i] = words;
    
    return pd.DataFrame(word_dict)

In [ ]:
topics_table(lda, count_vectorizer.get_feature_names())

Das Ergebnis sieht schon ziemlich gut aus. Abgesehen von dem ersten Topic (#00) sind alle ziemlich gut interpretierbar!

Wie viel Raum nehmen die einzelnen Topics in Anspruch? Das kannst du ausrechnen, indem du die Transformation durch das Topic Model vornehmen lässt und dadurch die Gewichte ermittelst. Die Summierung geht über die Achse, in der alle Dokumente mit ihren jeweiligen Anteilen am Topic enthalten haben. Um Prozentzahlen zu erhalten, musst du noch mit 100 multipltizieren. 

In [ ]:
W = lda.transform(count_vectors)
W.sum(axis=0)/W.sum()*100.0

Leider nummeriert pyLDAvis die Topics beginnend mit 1. Du kannst in der Darstellung erkennen, dass die Topics 5 und 8 die größten sind, das passt zu den Zahlen oben. Die Überlappungen sind nicht so einfach zu verstehen.

In [ ]:
import pyLDAvis.sklearn

lda_display = pyLDAvis.sklearn.prepare(lda, count_vectors, count_vectorizer, sort_topics=False)
pyLDAvis.display(lda_display)

## `pyLDAvis` verschafft dir einen guten Überblick

Mithilfe von `pyLDAvis` kannt du dir einen guten Überblick über die Struktur eines LDA-Modells verschaffen. Leider geht das auch nur für LDA-Modelle und nicht für NMF oder SVD. Spätestens hier hast du also ein Argument, das für LDA spricht.